In [2]:
import pandas as pd
import anndata

# Paths for the CSV files and the output .h5ad file
obs_file_path = "/content/drive/MyDrive/SingleCellModel/cell_barcde_labels.csv"
count_matrix_file_path = "/content/drive/MyDrive/SingleCellModel/brain_atlas_full_counts_table.csv"

obs_file_path = "./data/cell_barcde_labels.csv"
count_matrix_file_path = "./data/brain_atlas_full_counts_table.csv"

h5ad_file_path = "output_data.h5ad"


In [3]:

# # Define chunk size based on available memory and dataset dimensions
# chunk_size = 5000  # You might need to adjust this based on your specific dataset and memory constraints

# # Create an initial empty AnnData object and save to .h5ad
# # This step initializes the .h5ad file
# initial_adata = anndata.AnnData()
# initial_adata.write(h5ad_file_path)

# # Process data in chunks
# obs_iter = pd.read_csv(obs_file_path, chunksize=chunk_size, index_col=0)
# count_matrix_iter = pd.read_csv(count_matrix_file_path, chunksize=chunk_size, index_col=0)

# for obs_chunk, count_matrix_chunk in zip(obs_iter, count_matrix_iter):
#     # Create an AnnData object for the current chunk
#     chunk_adata = anndata.AnnData(X=count_matrix_chunk.values,
#                                   obs=obs_chunk,
#                                   var=pd.DataFrame(index=count_matrix_chunk.index))
    
#     # Append the chunk to the .h5ad file
#     chunk_adata.write(h5ad_file_path, append=True)

# print(f"Data has been saved to {h5ad_file_path}")



# try and sparsify the count matrix all at once and create a single anndata object

In [5]:

# import pandas as pd

# csv_file_path = 'path_to_input.csv'
# hdf5_file_path = 'path_to_output.h5'
# chunksize = 50  # Adjust based on your memory; this reads 5000 rows at a time

# # Use the first chunk to create the HDF5 file (with overwrite mode 'w')
# first_chunk = True

# # Read the CSV in chunks
# for chunk in pd.read_csv(csv_file_path, chunksize=chunksize):
#     if first_chunk:
#         chunk.to_hdf(hdf5_file_path, key='data', mode='w')
#         first_chunk = False
#     else:
#         chunk.to_hdf(hdf5_file_path, key='data', mode='a', format='table', append=True)



import numpy as np
from scipy.sparse import vstack, csr_matrix

# Parameters
filename = 'sparse_data.csv'
chunk_size = 10  # Adjust based on your memory capacity

# Initialize a list to store sparse matrices
sparse_chunks = []

# Calculate total number of rows in the CSV to determine loop iterations
with open(count_matrix_file_path, 'r') as f:
    num_lines = sum(1 for line in f) - 1  # Subtract 1 to exclude the header




# Read the CSV in chunks and convert to sparse matrices
for i in range(0, num_lines, chunk_size):
    chunk = np.genfromtxt(count_matrix_file_path, delimiter=',', skip_header=i+1, max_rows=chunk_size)
    sparse_chunk = csr_matrix(chunk)
    sparse_chunks.append(sparse_chunk)
    break

i


0

In [6]:
num_lines

36601

In [ ]:

# Combine the sparse matrices

sparse_matrix = vstack(sparse_chunks)


# Read chunks of the count matrix and append to an adata objet file.

In [2]:
import pandas as pd
import anndata
from scipy.sparse import csr_matrix, vstack, hstack
import numpy as np
from pathlib import Path

def append_chunk_to_h5ad(adata_chunk, h5ad_path):
    """Appends the chunk of data to the existing .h5ad file."""
    
    if h5ad_path.exists(): # os.path.exists(h5ad_path):
        adata_existing = anndata.read(h5ad_path)
        adata_combined = adata_existing.concatenate(adata_chunk, index_unique=None)
        adata_combined.write(h5ad_path)
    else:
        adata_chunk.write(h5ad_path)



# Paths
obs_file_path = "./data/cell_barcode_labels.csv"
count_matrix_file_path = "./data/brain_atlas_full_counts_table.csv"
h5ad_file_path = "./data/output_data.h5ad"



In [3]:

# Read the entire obs table

obs_ = pd.read_csv(obs_file_path,index_col=0)



In [4]:
import time
import dask.dataframe as dd
import dask.array as da

# Define the chunk size
chunk_size = 50  # Adjust this based on your memory capacity

# Create an empty list to store sparse matrices from each chunk
sparse_matrices = []
genes = []
chunk_counter = 0
st = time.time()
# 1. Read the CSV in chunks using a context manager
with pd.read_csv(count_matrix_file_path, chunksize=chunk_size) as reader:
    for chunk in reader:
        chunk_counter += 1
        print(f"chunk number {chunk_counter}")
        # 2. get the genes
        gene_chunk = chunk['genes']
        genes.append(gene_chunk)
        # 3. Convert the chunk to a sparse matrix and store
        sparse_chunk = csr_matrix(chunk.iloc[:,1:].values, dtype=np.uint8)
        sparse_matrices.append(sparse_chunk)

        lt = time.time() - st
        print(f'read #{chunk_counter}x  = {chunk.shape} in {lt:2f}s')

        st = time.time()


# 4. Combine the sparse matrices vertically
sparse_matrix = vstack(sparse_matrices)
genes_ = pd.concat(genes, axis=0)

adata = anndata.AnnData(X=sparse_matrix.transpose(),
                                obs=obs_,
                                var=pd.DataFrame(index=genes_))


adata.write(f"./data/chunk{chunk_size}_output_data.h5ad")
total_reads = sparse_matrix.shape[0] * sparse_matrix.shape[1]
print(f'read {chunk_size}x10 = {total_reads} in {etime:2f}s')
print(f'speed {chunk_size}= {total_reads/etime:2f} reads/s')


chunk number 1
read #1x  = (50, 713627) in 144.801319s
chunk number 2
read #2x  = (50, 713627) in 145.000706s
chunk number 3
read #3x  = (50, 713627) in 144.864834s
chunk number 4
read #4x  = (50, 713627) in 143.185682s
chunk number 5
read #5x  = (50, 713627) in 173.006080s
chunk number 6
read #6x  = (50, 713627) in 2143.327178s
chunk number 7
read #7x  = (50, 713627) in 143.198500s
chunk number 8
read #8x  = (50, 713627) in 143.815643s
chunk number 9
read #9x  = (50, 713627) in 144.206183s
chunk number 10
read #10x  = (50, 713627) in 144.942606s
chunk number 11
read #11x  = (50, 713627) in 143.810245s
chunk number 12
read #12x  = (50, 713627) in 143.988863s
chunk number 13
read #13x  = (50, 713627) in 143.070242s
chunk number 14
read #14x  = (50, 713627) in 143.015940s
chunk number 15
read #15x  = (50, 713627) in 141.978829s
chunk number 16
read #16x  = (50, 713627) in 141.454021s
chunk number 17
read #17x  = (50, 713627) in 140.904442s
chunk number 18
read #18x  = (50, 713627) in 140

KeyboardInterrupt: 

In [3]:
# count_chunk = count_matrix_iter.get_chunk().T

In [13]:
chunk_tests = [50, 100, 500, 1000, 5000]


import time

# Define the chunk size
chunk_size = 100  # Adjust this based on your memory capacity
for chunk_size in chunk_tests:
    # Create an empty list to store sparse matrices from each chunk
    sparse_matrices = []
    genes = []
    chunk_counter = 0
    st = time.time()
    # 1. Read the CSV in chunks using a context manager
    with pd.read_csv(count_matrix_file_path, chunksize=chunk_size) as reader:
        for chunk in reader:
            chunk_counter += 1
            print(f"chunk number {chunk_counter}")
            # 2. get the genes
            gene_chunk = chunk['genes']
            genes.append(gene_chunk)
            # 3. Convert the chunk to a sparse matrix and store
            sparse_chunk = csr_matrix(chunk.iloc[:,1:].values, dtype=np.uint8)
            sparse_matrices.append(sparse_chunk)
            if chunk_counter > 9:
                break

    # 3. Combine the sparse matrices vertically
    sparse_matrix = vstack(sparse_matrices)
    genes_ = pd.concat(genes, axis=0)

    adata = anndata.AnnData(X=sparse_matrix.transpose(),
                                    obs=obs_,
                                    var=pd.DataFrame(index=genes_))


    adata.write(f"./data/chunk{chunk_size}_output_data.h5ad")
    total_reads = sparse_matrix.shape[0] * sparse_matrix.shape[1]
    etime = time.time() - st
    print(f'read {chunk_size}x10 = {total_reads} in {etime:2f}s')
    print(f'speed {chunk_size}= {total_reads/etime:2f} reads/s')




chunk number 1
chunk number 2
chunk number 3
chunk number 4
chunk number 5
chunk number 6
chunk number 7
chunk number 8
chunk number 9
chunk number 10
read 50x10 = 356813000 in 1488.043364s
speed 50= 239786.694811 reads/s
chunk number 1
chunk number 2
chunk number 3
chunk number 4
chunk number 5
chunk number 6
chunk number 7
chunk number 8
chunk number 9
chunk number 10
read 100x10 = 713626000 in 2943.040450s
speed 100= 242479.168112 reads/s
chunk number 1
chunk number 2
chunk number 3
chunk number 4
chunk number 5
chunk number 6
chunk number 7
chunk number 8
chunk number 9
chunk number 10
read 500x10 = 3568130000 in 50581.309304s
speed 500= 70542.460231 reads/s


In [12]:
4800/748

6.4171122994652405

In [6]:

# 3. Combine the sparse matrices vertically
sparse_matrix = vstack(sparse_matrices)
genes_ = pd.concat(genes, axis=0)

adata = anndata.AnnData(X=sparse_matrix.transpose(),
                                  obs=obs_,
                                  var=pd.DataFrame(index=genes_))

adata.write(h5ad_file_path)



In [7]:
sparse_matrix.shape

(4800, 713626)

In [8]:
adata.shape



(713626, 4800)

In [11]:
genes_.shape

(4800,)

In [ ]:
sparse_chunk = scipy.sparse.csr_matrix(chunk.iloc[:,1:].values, dtype=np.uint8)

In [ ]:
sparse_chunk

<40x713626 sparse matrix of type '<class 'numpy.uint8'>'
	with 470307 stored elements in Compressed Sparse Row format>

In [ ]:
count_chunk.head()

genes,MIR1302-2HG,FAM138A,OR4F5,AL627309.1,AL627309.3,AL627309.2,AL627309.5,AL627309.4,AP006222.2,AL732372.1
GGCCTAATCGATTTAG-1_1,0,0,0,0,0,0,0,0,0,0
TAGTAACGTAGTCAAT-1_1,0,0,0,0,0,0,0,0,0,0
GAAAGCCAGCAGCTCA-1_1,0,0,0,0,0,0,0,0,0,0
ACTCACCTCCTCCCTC-1_1,0,0,0,0,0,0,0,0,0,0
CTTCATCCAATCGCAC-1_1,0,0,0,0,0,0,0,0,0,0


In [ ]:


tmp = csr_matrix(count_chunk.values)
var_ = pd.DataFrame(index=count_chunk.columns)

In [ ]:
count_chunk.shape, obs_.shape, var_.shape

((713626, 10), (713626, 3), (10, 0))

In [ ]:

adata_chunk = anndata.AnnData(X=csr_matrix(count_chunk.values),
                                  obs=obs_,
                                  var=pd.DataFrame(index=count_chunk.columns))

In [ ]:
adata_chunk.shape


(713626, 10)

In [ ]:


count_matrix_file_path = "./data/brain_atlas_full_counts_table.csv"
h5ad_file_path = "./data/output_data.h5ad"

# Define chunk size
chunk_size = 10  # Adjust based on memory constraints

# Read and process data in chunks
# obs_iter = pd.read_csv(obs_file_path, chunksize=chunk_size, index_col=0)
obs_ = pd.read_csv(obs_file_path,index_col=0)

count_matrix_iter = pd.read_csv(count_matrix_file_path, chunksize=chunk_size, index_col=0)


count_matrix_chunk = count_matrix_iter.get_chunk().T

# first chunk test
adata_chunk = anndata.AnnData(X=csr_matrix(count_matrix_chunk.values),
                                  obs=obs_,
                                  var=pd.DataFrame(index=count_matrix_chunk.columns))


NameError: name 'h5ad_path' is not defined

In [ ]:

adata_chunk.write(h5ad_file_path)




In [ ]:
adata_chunk.var

""
genes
MIR1302-2HG
FAM138A
OR4F5
AL627309.1
AL627309.3
AL627309.2
AL627309.5
AL627309.4
AP006222.2


In [ ]:

# second chunk test
count_matrix_chunk = count_matrix_iter.get_chunk().T




In [ ]:

adata_chunk2 = anndata.AnnData(X=csr_matrix(count_chunk.values),
                                  obs=obs_,
                                  var=pd.DataFrame(index=count_matrix_chunk.columns))
    


In [ ]:
adata_chunk.concatenate(adata_chunk2, index_unique=None)

/Users/ergonyc/miniforge3/envs/scverse10/lib/python3.10/site-packages/anndata/_core/anndata.py:1755: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(
/Users/ergonyc/miniforge3/envs/scverse10/lib/python3.10/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/ergonyc/miniforge3/envs/scverse10/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ergonyc/miniforge3/envs/scverse10/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_name

AttributeError: Can only use .str accessor with string values!

In [ ]:

"""Appends the chunk of data to the existing .h5ad file."""
if os.path.exists(h5ad_path):
    adata_existing = anndata.read(h5ad_path)
    adata_combined = adata_existing.concatenate(adata_chunk, index_unique=None)
    adata_combined.write(h5ad_path)
else:
        

        # Append the chunk to the .h5ad file
        append_chunk_to_h5ad(adata_chunk, h5ad_file_path)


        """Appends the chunk of data to the existing .h5ad file."""
        if os.path.exists(h5ad_path):
            adata_existing = anndata.read(h5ad_path)
            adata_combined = adata_existing.concatenate(adata_chunk, index_unique=None)
            adata_combined.write(h5ad_path)
        else:
            adata_chunk.write(h5ad_path)





In [ ]:


# Use the first chunk to create the HDF5 file (with overwrite mode 'w')
first_chunk = True


while True:
    try:

        count_matrix_chunk = count_matrix_iter.get_chunk()



        adata_chunk = anndata.AnnData(X=csr_matrix(count_chunk.values),
                                  obs=obs_chunk,
                                  var=pd.DataFrame(index=count_matrix_chunk.columns))
    
        # Append the chunk to the .h5ad file
        append_chunk_to_h5ad(adata_chunk, h5ad_file_path)


        """Appends the chunk of data to the existing .h5ad file."""
        if os.path.exists(h5ad_path):
            adata_existing = anndata.read(h5ad_path)
            adata_combined = adata_existing.concatenate(adata_chunk, index_unique=None)
            adata_combined.write(h5ad_path)
        else:
            adata_chunk.write(h5ad_path)




        if first_chunk:
            count_matrix_chunk.to_hdf(hdf5_file_path, key='data', mode='w')
            first_chunk = False
        else:
            count_matrix_chunk.to_hdf(hdf5_file_path, key='data', mode='a', format='table', append=True)
            


        adata_chunk = anndata.AnnData(X=count_matrix_chunk.values,
                                    obs=obs_chunk,
                                    var=pd.DataFrame(index=count_matrix_chunk.columns))
        
        # Append the chunk to the .h5ad file
        append_chunk_to_h5ad(adata_chunk, h5ad_file_path)

    except StopIteration:
        break
    # for obs_chunk, count_matrix_chunk in zip(obs_iter, count_matrix_iter):
    # Create an AnnData object for the current chunk

    
        # Read the CSV in chunks
    for chunk in pd.read_csv(csv_file_path, chunksize=chunksize):
    if first_chunk:
        chunk.to_hdf(hdf5_file_path, key='data', mode='w')
        first_chunk = False
    else:
        chunk.to_hdf(hdf5_file_path, key='data', mode='a', format='table', append=True)
        
    
    tmp = count_matrix_chunk.values    

    adata_chunk = anndata.AnnData(X=count_matrix_chunk.values,
                                  obs=obs_chunk,
                                  var=pd.DataFrame(index=count_matrix_chunk.columns))
    
    # Append the chunk to the .h5ad file
    append_chunk_to_h5ad(adata_chunk, h5ad_file_path)
    break
print(f"Data has been saved to {h5ad_file_path}")


In [ ]:
obs_chunk

NameError: name 'obs_chunk' is not defined

In [ ]:

# Define chunk size
chunk_size = 500  # Adjust based on memory constraints

# Read and process data in chunks
obs_iter = pd.read_csv(obs_file_path, chunksize=chunk_size, index_col=0)


FileNotFoundError: [Errno 2] No such file or directory: './data/cell_barcde_labels.csv'

In [120]:
import pandas as pd
import scipy.sparse

# Define the chunk size
chunk_size = 1024  # 

# Create an empty list to store sparse matrices from each chunk
sparse_matrices = []
genes = []
chunk_counter = 0
# 1. Read the CSV in chunks using a context manager
with pd.read_csv(count_matrix_file_path, nrows=50) chunksize=chunk_size) as reader:
    for chunk in reader:
        chunk_counter += 1
        print(f"chunk number {chunk_counter}")
        # 2. get the genes
        gene_chunk = chunk['gene']


        
        genes.append(gene_chunk)
        # 3. Convert the chunk to a sparse matrix and store
        sparse_chunk = scipy.sparse.csr_matrix(chunk.iloc[:,1:].values, dtype=np.uint8)
        sparse_matrices.append(sparse_chunk)


# 3. Combine the sparse matrices vertically
sparse_matrix = scipy.sparse.vstack(sparse_matrices)

chunk number 1


TypeError: no supported conversion for types: (dtype('O'),)

In [130]:
sparse_chunk = scipy.sparse.csr_matrix(chunk.iloc[:,1:].values, dtype=np.uint8)

In [131]:
sparse_chunk

<40x713626 sparse matrix of type '<class 'numpy.uint8'>'
	with 470307 stored elements in Compressed Sparse Row format>

In [108]:
count_chunk.head()

genes,MIR1302-2HG,FAM138A,OR4F5,AL627309.1,AL627309.3,AL627309.2,AL627309.5,AL627309.4,AP006222.2,AL732372.1
GGCCTAATCGATTTAG-1_1,0,0,0,0,0,0,0,0,0,0
TAGTAACGTAGTCAAT-1_1,0,0,0,0,0,0,0,0,0,0
GAAAGCCAGCAGCTCA-1_1,0,0,0,0,0,0,0,0,0,0
ACTCACCTCCTCCCTC-1_1,0,0,0,0,0,0,0,0,0,0
CTTCATCCAATCGCAC-1_1,0,0,0,0,0,0,0,0,0,0


In [97]:
obs_iter = pd.read_csv(obs_file_path, chunksize=chunk_size, index_col=0)
obs_chunk = obs_iter.get_chunk()

obs_ = pd.read_csv(obs_file_path,index_col=0)


tmp = csr_matrix(count_chunk.values)
var_ = pd.DataFrame(index=count_chunk.columns)

In [98]:
count_chunk.shape, obs_.shape, var_.shape

((713626, 10), (713626, 3), (10, 0))

In [99]:

adata_chunk = anndata.AnnData(X=csr_matrix(count_chunk.values),
                                  obs=obs_,
                                  var=pd.DataFrame(index=count_chunk.columns))

In [109]:
adata_chunk.shape


(713626, 10)

In [110]:


count_matrix_file_path = "./data/brain_atlas_full_counts_table.csv"
h5ad_file_path = "./data/output_data.h5ad"

# Define chunk size
chunk_size = 10  # Adjust based on memory constraints

# Read and process data in chunks
# obs_iter = pd.read_csv(obs_file_path, chunksize=chunk_size, index_col=0)
obs_ = pd.read_csv(obs_file_path,index_col=0)

count_matrix_iter = pd.read_csv(count_matrix_file_path, chunksize=chunk_size, index_col=0)


count_matrix_chunk = count_matrix_iter.get_chunk().T

# first chunk test
adata_chunk = anndata.AnnData(X=csr_matrix(count_matrix_chunk.values),
                                  obs=obs_,
                                  var=pd.DataFrame(index=count_matrix_chunk.columns))


NameError: name 'h5ad_path' is not defined

In [111]:

adata_chunk.write(h5ad_file_path)




In [113]:
adata_chunk.var

""
genes
MIR1302-2HG
FAM138A
OR4F5
AL627309.1
AL627309.3
AL627309.2
AL627309.5
AL627309.4
AP006222.2


In [115]:

# second chunk test
count_matrix_chunk = count_matrix_iter.get_chunk().T




In [116]:

adata_chunk2 = anndata.AnnData(X=csr_matrix(count_chunk.values),
                                  obs=obs_,
                                  var=pd.DataFrame(index=count_matrix_chunk.columns))
    


In [117]:
adata_chunk.concatenate(adata_chunk2, index_unique=None)

/Users/ergonyc/miniforge3/envs/scverse10/lib/python3.10/site-packages/anndata/_core/anndata.py:1755: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(
/Users/ergonyc/miniforge3/envs/scverse10/lib/python3.10/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/ergonyc/miniforge3/envs/scverse10/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ergonyc/miniforge3/envs/scverse10/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_name

AttributeError: Can only use .str accessor with string values!

In [ ]:

"""Appends the chunk of data to the existing .h5ad file."""
if os.path.exists(h5ad_path):
    adata_existing = anndata.read(h5ad_path)
    adata_combined = adata_existing.concatenate(adata_chunk, index_unique=None)
    adata_combined.write(h5ad_path)
else:
        

        # Append the chunk to the .h5ad file
        append_chunk_to_h5ad(adata_chunk, h5ad_file_path)


        """Appends the chunk of data to the existing .h5ad file."""
        if os.path.exists(h5ad_path):
            adata_existing = anndata.read(h5ad_path)
            adata_combined = adata_existing.concatenate(adata_chunk, index_unique=None)
            adata_combined.write(h5ad_path)
        else:
            adata_chunk.write(h5ad_path)





In [ ]:


# Use the first chunk to create the HDF5 file (with overwrite mode 'w')
first_chunk = True


while True:
    try:

        count_matrix_chunk = count_matrix_iter.get_chunk()



        adata_chunk = anndata.AnnData(X=csr_matrix(count_chunk.values),
                                  obs=obs_chunk,
                                  var=pd.DataFrame(index=count_matrix_chunk.columns))
    
        # Append the chunk to the .h5ad file
        append_chunk_to_h5ad(adata_chunk, h5ad_file_path)


        """Appends the chunk of data to the existing .h5ad file."""
        if os.path.exists(h5ad_path):
            adata_existing = anndata.read(h5ad_path)
            adata_combined = adata_existing.concatenate(adata_chunk, index_unique=None)
            adata_combined.write(h5ad_path)
        else:
            adata_chunk.write(h5ad_path)




        if first_chunk:
            count_matrix_chunk.to_hdf(hdf5_file_path, key='data', mode='w')
            first_chunk = False
        else:
            count_matrix_chunk.to_hdf(hdf5_file_path, key='data', mode='a', format='table', append=True)
            


        adata_chunk = anndata.AnnData(X=count_matrix_chunk.values,
                                    obs=obs_chunk,
                                    var=pd.DataFrame(index=count_matrix_chunk.columns))
        
        # Append the chunk to the .h5ad file
        append_chunk_to_h5ad(adata_chunk, h5ad_file_path)

    except StopIteration:
        break
    # for obs_chunk, count_matrix_chunk in zip(obs_iter, count_matrix_iter):
    # Create an AnnData object for the current chunk

    
        # Read the CSV in chunks
    for chunk in pd.read_csv(csv_file_path, chunksize=chunksize):
    if first_chunk:
        chunk.to_hdf(hdf5_file_path, key='data', mode='w')
        first_chunk = False
    else:
        chunk.to_hdf(hdf5_file_path, key='data', mode='a', format='table', append=True)
        
    
    tmp = count_matrix_chunk.values    

    adata_chunk = anndata.AnnData(X=count_matrix_chunk.values,
                                  obs=obs_chunk,
                                  var=pd.DataFrame(index=count_matrix_chunk.columns))
    
    # Append the chunk to the .h5ad file
    append_chunk_to_h5ad(adata_chunk, h5ad_file_path)
    break
print(f"Data has been saved to {h5ad_file_path}")


In [17]:
obs_chunk

NameError: name 'obs_chunk' is not defined

In [10]:

# Define chunk size
chunk_size = 500  # Adjust based on memory constraints

# Read and process data in chunks
obs_iter = pd.read_csv(obs_file_path, chunksize=chunk_size, index_col=0)


FileNotFoundError: [Errno 2] No such file or directory: './data/cell_barcde_labels.csv'